## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

Fri Aug 16 21:22:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()
from IPython.display import display, Image

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.7/78.2 GB disk)


## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Custom Dataset

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eedAtIN3PCizBbvdcTIy")
project = rf.workspace("kripansh").project("grey-scientific-labs-pxumf")
version = project.version(4)
dataset = version.download("yolov5-obb")


mkdir: cannot create directory ‘/content/datasets’: File exists
/content/datasets
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os

# Define the classes and their corresponding indices
class_names = ['House', 'Tennis-court']
class_indices = {name: index for index, name in enumerate(class_names)}


# Image dimensions
img_width, img_height = 640, 640

# Path to the folder containing the annotation files
folder_path = '/content/datasets/Grey-Scientific-Labs-4/train/labels'

# Function to normalize coordinates
def normalize_coordinates(coord, max_value):
    return float(coord) / max_value

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()


        new_lines = []
        for line in lines:
            parts = line.strip().split(' ')
            if len(parts) == 10:
                # Extract label and coordinates
                label = parts[-2]
                coords = parts[:8]

                # Get the class index
                class_index = class_indices.get(label, -1)
                if class_index != -1:
                    # Normalize coordinates
                    normalized_coords = [normalize_coordinates(coords[i], img_width if i % 2 == 0 else img_height) for i in range(8)]

                    # Convert to the desired format and add to new lines
                    new_line = f"{class_index} " + " ".join(map(str, normalized_coords))
                    new_lines.append(new_line)

        # Write the converted lines to a new file or overwrite the existing file
        with open(file_path, 'w') as file:
            file.write('\n'.join(new_lines))

In [ ]:
import os

# Define class names and their corresponding indices
class_mapping = {
    'House': 0,
    'Tennis-court': 1
}

# Path to the labels directory
label_dir = '/content/datasets/Grey-Scientific-Labs-4/valid/labels/'

# Process each label file
for label_file in os.listdir(label_dir):
    label_path = os.path.join(label_dir, label_file)
    with open(label_path, 'r') as file:
        lines = file.readlines()

    corrected_lines = []
    for line in lines:
        parts = line.strip().split()
        if parts[0] in class_mapping:
            # Replace class name with class ID
            parts[0] = str(class_mapping[parts[0]])
            corrected_lines.append(" ".join(parts))

    # Overwrite the label file with corrected data
    with open(label_path, 'w') as file:
        file.write("\n".join(corrected_lines))


## Custom Training

In [ ]:
%cd {HOME}

!yolo task=obb mode=train model=yolov8s-obb.pt data={dataset.location}/data.yaml epochs=35 imgsz=640

/content
Ultralytics YOLOv8.2.78 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8s-obb.pt, data=/content/datasets/Grey-Scientific-Labs-4/data.yaml, epochs=35, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/val/confusion_matrix.png', width=600)

/content


FileNotFoundError: [Errno 2] No such file or directory: '/content/runs/obb/val/confusion_matrix.png'

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train4/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/obb/train4/val_batch0_pred.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=obb mode=val model={HOME}/runs/obb/train4/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=obb mode=predict model={HOME}/runs/obb/train/weights/best.pt conf=0.5 source={dataset.location}/test/images save=true

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/obb/predict/*.jpg'):
      display(Image(filename=image_path, height=600))
      print("\n")